## Embedding with keras

1. [Neural Network Embeddings Explained](https://towardsdatascience.com/neural-network-embeddings-explained-4d028e6f0526)
2. [How to Use Word Embedding Layers for Deep Learning with Keras
](https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/?source=post_page---------------------------)

In [40]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

### 1. 创建文档和标签

In [41]:
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = np.array([1,1,1,1,1,0,0,0,0,0])

### 2. 进行 one-hot 独热编码

我们假设 Vocabulary 的大小为 50, 这远比我们给的文件档的单词要多。但是这样可以降低 hash 冲突的可能性。

one-hot hash 冲突？

Keras 的 `one_hot` 函数内部使用是 `hashing_trick` 函数对 text 进行 hash。hash 可能会有冲突，并不保证惟一。


In [42]:
print(one_hot('a c aa', 8))
print(one_hot('a c aa', 5)) # 冲突
print(one_hot('a c aa', 3)) # 完全冲突

[1, 2, 5]
[2, 4, 2]
[2, 2, 2]


In [43]:
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

[[45, 38], [35, 49], [6, 11], [24, 49], [33], [8], [19, 11], [32, 35], [19, 49], [26, 32, 38, 22]]


这样我们每个文档中的单词都对应的进行 code 了。但是文档中的长度并不一样，
我们可以使用 `pad_sequnces()` 函数来进行对齐。

In [44]:
# pad_documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length,
                        padding='post')
print(padded_docs)

[[45 38  0  0]
 [35 49  0  0]
 [ 6 11  0  0]
 [24 49  0  0]
 [33  0  0  0]
 [ 8  0  0  0]
 [19 11  0  0]
 [32 35  0  0]
 [19 49  0  0]
 [26 32 38 22]]


现在就可以使用 Embedding layer 了。我们需要将一个大小为 50 的 Vocabulary, 每个 input 长度为 4 的样本，嵌入到一个 8 维的 Embedding 空间内。

In [45]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy',
             metrics=['acc'])

# summarize the model
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten_2 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None


最后进行，模型拟合和评测。

In [46]:
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)

# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy * 100))

Accuracy: 89.999998


参考资料给出的准确率是 100%, 但我的为什么只有 89% 那？

现在可以将我们从 Embedding layer 学习到参数保存下来。

## 使用预先生成的 Embedding Space 

在 Keras 的 Embedding Layer 中我们可以使用从其它模型中学习到的 Embedding 空间。

在 NLP 处理，使用已有的 embedding 是很常见的。

例如，我们可以使用 [GloVe: Global Vectors for Word Representation](https://nlp.stanford.edu/projects/glove/) 的 Embedding.